In [26]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('dataset.csv')
df = df[~df.applymap(lambda x: isinstance(x, str) and x.isalpha()).any(axis=1)]
cols_to_change = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'target']
df[cols_to_change] = df[cols_to_change].astype(float)
X = df.drop('target', axis=1)
y = df['target']
# Scale numerical features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [27]:
import torch.nn as nn

# Define the model
class SimpleBinaryClassifier(nn.Module):
    def __init__(self):
        super(SimpleBinaryClassifier, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        ])
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [28]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Split dataset into train and test sets
dataset = TensorDataset(X_tensor, y_tensor)
train_dataset, test_dataset = random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleBinaryClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [29]:
n_epochs = 50
for epoch in range(n_epochs):
    # Training
    train_loss = 0.0
    train_acc = 0.0
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target.view(-1,1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = torch.round(output)
        train_acc += torch.sum(pred == target.view(-1,1)).item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)

    # Testing
    test_loss = 0.0
    test_acc = 0.0
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.view(-1,1))
            test_loss += loss.item() * data.size(0)
            pred = torch.round(output)
            test_acc += torch.sum(pred == target.view(-1,1)).item()
            
    test_loss = test_loss / len(test_loader.dataset)
    test_acc = test_acc / len(test_loader.dataset)
    # Print results for each epoch
    # print(f'Epoch {epoch+1}/{n_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
print(f'Final Test Accuracy: {test_acc:.4f}')

Final Test Accuracy: 0.7500


In [21]:
import torch.nn as nn

# Define the model
class SimpleBinaryClassifierWithDropout(nn.Module):
    def __init__(self, dropout):
        super(SimpleBinaryClassifierWithDropout, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(7, 128),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        ])
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [25]:
# Define loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
dropout_rate = [0.1,0.3,0.5,0.7]

for dropout in dropout_rate:
    model = SimpleBinaryClassifierWithDropout(dropout).to(device)
    n_epochs = 50
    for epoch in range(n_epochs):
        # Training
        train_loss = 0.0
        train_acc = 0.0
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.view(-1,1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            pred = torch.round(output)
            train_acc += torch.sum(pred == target.view(-1,1)).item()
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_acc / len(train_loader.dataset)

        # Testing
        test_loss = 0.0
        test_acc = 0.0
        model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target.view(-1,1))
                test_loss += loss.item() * data.size(0)
                pred = torch.round(output)
                test_acc += torch.sum(pred == target.view(-1,1)).item()
                
        test_loss = test_loss / len(test_loader.dataset)
        test_acc = test_acc / len(test_loader.dataset)
        
        # Print results for each epoch
        # print(f'Epoch {epoch+1}/{n_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
    
# Print final test accuracy for each dropout rate
print(f'Dropout rate: {dropout:.1f}, Final Test Accuracy: {test_acc:.4f}')

Dropout rate: 0.7, Final Test Accuracy: 0.5066
